In [ ]:
import os

os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'

In [ ]:
import warnings

warnings.filterwarnings('ignore')

查看pipeline支持的任务类型

In [ ]:
from transformers.pipelines import SUPPORTED_TASKS
for k, v in SUPPORTED_TASKS.items():
    print(k, v)

pipeline创建与使用

In [ ]:
from transformers import *

根据任务类型直接创建Pipeline

In [ ]:
pipe = pipeline("text-classification")

In [ ]:
pipe("very good!")

指定任务类型，再指定模型，创建基于指定模型的Pipeline

In [ ]:
pipe = pipeline("text-classification",model="uer/roberta-base-finetuned-dianping-chinese")

In [ ]:
pipe("还不错")

预先加载模型，再创建Pipeline

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")
tokenizer = AutoTokenizer.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")
pipe = pipeline("text-classification",model=model,tokenizer=tokenizer)

In [ ]:
pipe("有点差劲")

In [ ]:
pipe.model.device

In [ ]:
import torch
import time
times = []
for i in range(100):
    torch.cuda.synchronize()
    start = time.time()
    pipe("我觉得不太行！")
    torch.cuda.synchronize()
    end = time.time()
    times.append(end - start)
print(sum(times)/100)

In [ ]:
pipe = pipeline("text-classification",model="uer/roberta-base-finetuned-dianping-chinese", device=0)

In [ ]:
import torch
import time
times = []
for i in range(100):
    torch.cuda.synchronize()
    start = time.time()
    pipe("我觉得不太行！")
    torch.cuda.synchronize()
    end = time.time()
    times.append(end - start)
print(sum(times)/100)

In [ ]:
qa_pipeline = pipe = pipeline("question-answering",model="uer/roberta-base-finetuned-dianping-chinese", device=0)

In [ ]:
qa_pipeline(question="中国首都是哪里？", context="中国的首都：北京")

其他pipeline示例

In [ ]:
checkpoint = "google/owlvit-base-patch32"
detector = pipeline(model=checkpoint, task="zero-shot-object-detection")

In [ ]:
import requests
from PIL import Image

url = "https://unsplash.com/photos/oj0zeY2Ltk4/download?ixid=MnwxMjA3fDB8MXxzZWFyY2h8MTR8fHBpY25pY3xlbnwwfHx8fDE2Nzc0OTE1NDk&force=true&w=640"
im = Image.open(requests.get(url, stream=True).raw)
im

In [ ]:
predictions = detector(
    im,
    candidate_labels=["hat", "sunglasses", "book"],
)
predictions

In [ ]:
from PIL import ImageDraw

draw = ImageDraw.Draw(im)

for prediction in predictions:
    box = prediction["box"]
    label = prediction["label"]
    score = prediction["score"]
    xmin, ymin, xmax, ymax = box.values()
    draw.rectangle((xmin, ymin, xmax, ymax), outline="red", width=1)
    draw.text((xmin, ymin), f"{label}: {round(score,2)}", fill="red")

im

In [ ]:
from transformers import *
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")
model = AutoModelForSequenceClassification.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")

In [ ]:
input_text = "我觉得不太行！"
inputs = tokenizer(input_text, return_tensors="pt")
inputs

In [ ]:
res = model(**inputs)
res

In [ ]:
logits = res.logits
logits = torch.softmax(logits, dim=-1)
logits

In [ ]:
pred = torch.argmax(logits).item()
pred

In [ ]:
model.config.id2label

In [ ]:
result = model.config.id2label.get(pred)
result